In [23]:
# Import dependencies
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time, requests, os

In [2]:
# Yahoo Finance BaseURL
url = "https://finance.yahoo.com/"

# List of companies
companies = ["Tesla", "GameStop", "Microsoft"]

In [4]:
# Start chrome webdriver
driver = webdriver.Chrome(executable_path = r"C:\temp\chromedriver.exe")

# Hit url and wait for 2 seconds
driver.get(url)
time.sleep(2)

In [6]:
# Enter name of company in searchbox, and wait for 2 seconds
driver.find_element_by_xpath("//input[@placeholder = 'Search for news, symbols or companies']").send_keys("Tesla")
time.sleep(2)

# Click on Search icon and wait for 2 seconds
driver.find_element_by_xpath("//button[@id= 'header-desktop-search-button']").click()
time.sleep(2)

In [7]:
# Driver clicks on Historical Data tab and sleeps for 2 seconds
driver.find_element_by_xpath("//span[text() = 'Historical Data']").click()
time.sleep(2)

In [8]:
# Webpage uses dynamic loading of data where data loads upon scrolling down
# Driver scrolls down three times to load full table
for i in range(0,3):
 driver.execute_script("window.scrollBy(0,5000)")
 time.sleep(2)

In [29]:
# Fetch webpage and store in a variable
webpage = driver.page_source
# Print the fetched webpage
#print(webpage)

In [12]:
# Parse web page fetched from driver using Beautiful Soup
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [13]:
# Table is searched using class and stored in another variable
table = soup.find('table', class_='W(100%) M(0)')

In [14]:
# List of all the rows of stock data
rows = table.find_all('tr', class_='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)')

In [24]:
# Create empty list to store data
extracted_data = []

# Loop through each row of the table
for i in range(0, len(rows)):
    try:
        # Create empty dictionary to store present data in each row
        # Date, Open, High, Low, Close, Adj Close, Volume
        row_dict = {}
        # Extract all columns of the row and store in variable
        values = rows[i].find_all('td')

        # Values are extracted and stored in dictionary
        if len(values) == 7:
            row_dict["Date"] = values[0].find('span').text.replace(',', '')
            row_dict["Open"] = values[1].find('span').text.replace(',', '')
            row_dict["High"] = values[2].find('span').text.replace(',', '')
            row_dict["Low"] = values[3].find('span').text.replace(',', '')
            row_dict["Close"] = values[4].find('span').text.replace(',', '')
            row_dict["Adj Close"] = values[5].find('span').text.replace(',', '')
            row_dict["Volume"] = values[6].find('span').text.replace(',', '')

            # Append into list
            extracted_data.append(row_dict)
    
    except:
        # Check exception
        print("Row Number: " + str(i))

    finally:
        i = i + 1

In [25]:
row_dict

{'Date': 'Feb 06 2020',
 'Open': '139.98',
 'High': '159.17',
 'Low': '137.40',
 'Close': '149.79',
 'Adj Close': '149.79',
 'Volume': '199404000'}

In [28]:
# Convert list of dictionaries to a dataframe
extracted_data_df = pd.DataFrame(extracted_data)
extracted_data_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,Feb 05 2021,845.00,864.77,838.97,852.23,852.23,18524800
1,Feb 04 2021,855.00,856.50,833.42,849.99,849.99,15812700
2,Feb 03 2021,877.02,878.08,853.06,854.69,854.69,18343500
3,Feb 02 2021,844.68,880.50,842.20,872.79,872.79,24346200
4,Feb 01 2021,814.29,842.00,795.56,839.81,839.81,25391400
...,...,...,...,...,...,...,...
248,Feb 12 2020,155.57,157.95,152.67,153.46,153.46,60112500
249,Feb 11 2020,153.76,156.70,151.60,154.88,154.88,58487500
250,Feb 10 2020,160.00,164.00,150.48,154.26,154.26,123446000
251,Feb 07 2020,146.11,153.95,146.00,149.61,149.61,85317500
